In [1]:
import pandas as pd
import numpy as np
import pickle

In [2]:
def chunks(lst, n):
    chunked = [ lst[i:i + n] for i in range(0, len(lst), n) ]
    return np.array(chunked[:-1])

In [3]:
n = 20
normalize = True
device = "phone" # phone watch

data = {}

for i, person in enumerate(range(1600, 1650+1)):
    fname1 = f"../../raw/WISDM/{device}/accel/data_{person}_accel_{device}.txt"
    df1 = pd.read_csv(fname1, header=None, comment=";")
    df1 = df1.sort_values(2)
    
    fname2 = f"raw/{device}/gyro/data_{person}_gyro_{device}.txt"
    df2 = pd.read_csv(fname2, header=None, comment=";")
    df2 = df2.sort_values(2)
    
    df = pd.merge_asof(df1, df2, on=2, direction="backward").dropna()
    df = df.drop(columns=["0_x", "0_y", 2, "1_y"])
    
    values = df.values[:, 1:].astype(float)
    labels = df.values[:, 0]
    
    for c_int, c_str in enumerate("ABCDEFGHIJKLMOPQRS"):
        labels[labels == c_str] = c_int
    
    if normalize:
        values = (values - values.mean(axis=0)) / values.std(axis=0)
    
    dico = {}
    for c in set(labels):
        values_c = values[labels == c]
        X = np.array([ arr.T for arr in chunks(values_c, n) ])
        dico[c] = X
    
    data[i] = dico

In [4]:
with open(f"data_{device}.pickle", 'wb') as handle: 
    pickle.dump(data, handle)

In [5]:
with open(f"data_{device}.pickle", 'rb') as handle:
    data = pickle.load(handle)
    print( data.keys() )
    print( data[2].keys() )

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])
dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17])
